In [1]:
import numpy as np
import matplotlib.pyplot as plt
from fun import *

In [2]:
gammar = 1.4 
Nx = 600
k = 2
NumGLP = 5
dimPk = k + 1
NumEq = 3
CFL = 0.15
flux_type = 1
Limit_type = 2

In [3]:
# 获取高斯积分点和权重
point, weight = get_GLP(NumGLP)

In [4]:
# 网格
xa = 0
xb = 1
hx = (xb - xa)/Nx
h1 = 0.5*hx
x = np.linspace(xa, xb, Nx+1)
xc = np.linspace(xa + h1, xb - h1, Nx)

# 初始场
U = np.zeros((NumEq, Nx, NumGLP)) # [rho, rho*u, E]
for i in range(Nx):
    for j in range(NumGLP):
        x = xc[i] + h1 * point[j]
        U[0,i,j] = rho0(x)
        U[1,i,j] = rho0(x) * u0(x)
        U[2,i,j] = p0(x) / (gammar -1) + 0.5 * rho0(x) * u0(x) * u0(x)

In [5]:
# 基函数在高斯点的值和导数
# 基函数定义为 1, X, X^2-1/3, ...
#  -1 < X < 1
phi_gauss = np.zeros((NumGLP, dimPk))
phi_grad_gauss = np.zeros((NumGLP, dimPk))
phi_r = np.zeros(dimPk)
phi_l = np.zeros(dimPk)
mass = np.asarray([1, 1/3, 4/45])
for i in range(NumGLP):
    phi_gauss[i,0] = 1
    phi_grad_gauss[i,0] = 0

    phi_gauss[i,1] = point[i]
    phi_grad_gauss[i,1] = 1 / h1

    phi_gauss[i,2] = point[i] ** 2 - 1/3
    phi_grad_gauss[i,2] = 2 * point[i] / h1

phi_r[0] = 1; phi_r[1] = -1; phi_r[2] = 2 / 3
phi_l[0] = 1; phi_l[1] = 1; phi_l[2] = 2 / 3

In [6]:
# 将初始场投影到基函数上，获取基函数系数初始值
uh = np.zeros((NumEq, Nx, dimPk))
weight_matrix = np.diag(weight)
# for i in range(Nx):
#     for j in range(dimPk):
#         for k in range(NumGLP):
#             for m in range(NumEq):
#                 uh[m,i,j] += 0.5 * weight_matrix[k,k] * U[m,i,k] * phi_gauss[k,j]
for m in range(NumEq):
    uh[m] = 0.5 * U[m] @ weight_matrix @ phi_gauss
for d in range(dimPk):
    uh[:,:,d] /= mass[d]

In [7]:
# 计算波速alpha
alpha = 1
for i in range(Nx):
    alpha1, alpha2 = wavespeed(uh[:,i,0])
    alpha = max(alpha, alpha1, alpha2)
dt = CFL * hx / alpha